# Sprint 9 - Introducción al Machine Learning

## Introducción 


Para este proyecto, trabajamos para la compañía móvil Megaline, la cual no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Para esta tarea de clasificación se creará un modelo que escoja el plan correcto. Las condicionantes son desarrollar un modelo con la mayor exactitud posible, para el cual el umbral de exactitud es 0,75.

## Inicialización

Iniciamos con la importación de las librerías requeridas para el proyecto.

In [76]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Carga de Datos

Continuamos con la lectura de dataset, revisando que el separador por defecto sea efectivamente el que necesitamos. Para ello, imprimimos una muestra.

In [2]:
df = pd.read_csv("/datasets/users_behavior.csv")
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
2252,133.0,949.36,0.0,19036.43,1
2688,57.0,371.55,16.0,12283.46,0
1332,67.0,506.16,0.0,23867.03,1
1379,77.0,480.37,33.0,20586.08,0
1445,49.0,338.00,13.0,16963.20,0


In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [4]:
df.duplicated().sum()

0

En el enunciado nos señalan que los datos ya están procesados. Por lo que vemos, no habría ninguna columna con datos ausentes ni datos repetidos. Siendo ese el caso iremos directamente a la creación del modelo.

## Modelos de ML

Para los diferentes modelos de ML a utilizar, comenzaremos por la segmentación de datos que nos permita obtener nuestros conjuntos de entrenamiento, de validación y uno de prueba.

Para efectuar esto, necesitaremos ejecutar la funcion train_test_split dos veces. Posterior a aquello, continuaremos con nuestra tarea de clasificación.

### Segmentación del Dataset

Para el dataset de entrenamiento dejaremos 60% del dataframe total, para el de validación y prueba un 20%

In [5]:
#generamos una funcion para dividir nuestro dataset dos veces usando la funcion train_test_split

def split_train_val_test(df_input, stratify_colname='y', frac_train=0.6, frac_val=0.2, frac_test=0.2, 
                         random_state=12345):
    

    if frac_train + frac_val + frac_test != 1.0: #verifica que la suma de las particiones de 1
        raise ValueError('Los elementos %f, %f, %f no suman 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s no es una columna en el dataframe' % (stratify_colname))

    X = df_input # Posee todas las columnas
    y = df_input[[stratify_colname]] # la columna en la cual basamos el analisis

    # Divide el dataframe en porciones temporales
    df_train, df_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Divide finalmente el dataframe utilizando las porciones temporales para obtener las 3 partes
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp, y_temp, stratify=y_temp, 
                                                      test_size=relative_frac_test, random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test) #Verifica la integridad de los datos

    return df_train, df_val, df_test

In [6]:
df_train, df_val, df_test = split_train_val_test(df, stratify_colname='is_ultra', 
                                                 frac_train=0.60, frac_val=0.20, frac_test=0.20)

Verificando que el dataset haya sido dividido correctamente, utilizamos el atributo Shape para visualizar sus filas y columnas. Sumando sus filas nos da 3214 que es el mismo número total de nuestro dataset completo.

In [7]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


Para crear los elementos necesarios para nuestros modelos, tenemos lo siguiente:

In [8]:
features_train = df_train.drop("is_ultra", axis = 1)
target_train = df_train["is_ultra"]
features_valid = df_val.drop("is_ultra", axis = 1)
target_valid = df_val["is_ultra"]
features_test = df_test.drop("is_ultra", axis = 1)
target_test = df_test["is_ultra"]

### Modelo Árbol de Decisión

Para el modelo árbol de decisión, haremos una modificación en sus hiperparámetros respecto a su profundidad, dejandola entre 1 y 5 para verificar con qué valor de aquellos nos quedamos.

In [15]:
tree_score = 0
best_depth = 0
for a in range(1,6):
    model = DecisionTreeClassifier(random_state=1702, max_depth = a)
    model.fit(features_train, target_train)
    score = model.score(features_train, target_train)
    valid_score = model.score(features_valid, target_valid)
    if valid_score > tree_score:
        tree_score = valid_score
        best_depth = a
print("Profundidad =", best_depth, ":", tree_score, ", Calidad conjunto entrenamiento:", score)
test_score = model.score(features_test, target_test)
print("Calidad del conjunto de prueba:", test_score)
    


Profundidad = 5 : 0.7853810264385692 , Calidad conjunto entrenamiento: 0.8137966804979253
Calidad del conjunto de prueba: 0.8102643856920684


Del resultado anterior, tenemos que para el modelo de árbol de decisión tenemos una calidad de 0.81 en nuestros datos de entrenamiento, con una profundidad de 5 para los árboles. Por el lado del conjunto de validación tenemos una exactitud de 0.785 lo cual es normal ya que el modelo se entrena con sus datos, por ende claramente posee una mayor exactitud que en dataset de validación.

Para la exactitud del conjunto de prueba tenemos un valor de 0.81, bastante bueno para el modelo.

Pasaremos ahora a revisar otros modelos.

### Modelo Bosque Aleatorio

In [16]:
best_score = 0
best_est = 0

for est in range(1, 6): # selecciona el rango del hiperparámetro
    model2 = RandomForestClassifier(random_state=1702, n_estimators=est) # configura el número de árboles
    model2.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    score2 = model2.score(features_train, target_train)
    valid_score2 = model2.score(features_valid, target_valid)
    if valid_score2 > best_score:
        best_score = valid_score2 #guarda el mejor puntaje o calidad
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud
       
print("La exactitud en conjunto de validación es (n_estimators = ", best_est,"):",best_score)
print("La calidad del conjunto de entrenamiento es: ", score2)
test_score2 = model2.score(features_test, target_test)
print("La calidad del conjunto de prueba es: ", test_score2)

La exactitud en conjunto de validación es (n_estimators =  5 ): 0.7698289269051322
La calidad del conjunto de entrenamiento es:  0.9631742738589212
La calidad del conjunto de prueba es:  0.7729393468118196


Para este modelo, tenemos algo bastante peculiar, pero dentro de los parámetros normales en un modelo. El conjunto de entrenamiento logró una exactitud bastante alta, pero esta decae notoriamente al ser comparada con el conjunto de validación y el de prueba. Esto puede indicar que el modelo está sobreajustado.

Eso sí, este modelo cumple con nuestros estándares de selección respecto al umbral de calidad, por ende es un posible candidato a ser el seleccionado.

### Modelo Regresión Logística

In [11]:
model3 = LogisticRegression(random_state=1702, solver= 'liblinear')
model3.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model3.score(features_train, target_train) # calcula la exactitud en el conjunto de entrenamiento
score_valid = model3.score(features_valid, target_valid)
score_test = model3.score(features_test, target_test) # calcula la exactitud en el conjunto de prueba

print("Exactitud del modelo en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo en el conjunto de validación:", score_valid)
print("Exactitud del modelo en el conjunto de prueba:", score_test)

Exactitud del modelo en el conjunto de entrenamiento: 0.7105809128630706
Exactitud del modelo en el conjunto de validación: 0.71850699844479
Exactitud del modelo en el conjunto de prueba: 0.7107309486780715


Para este modelo, tenemos que en los 3 dataset nos da una exactitud de 0.71, lo cual nos deja bajo el umbral solicitado en nuestro proyecto, por lo que descartaremos este modelo como el mejor.

### Selección del mejor Modelo

Con nuestros datos de entrenamiento ya ejecutados en los 3 modelos, tenemos que decidir cuál de ellos es el más adecuado para seleccionarlo como el mejor. Ya señalamos anteriormente que el modelo de regresión logística queda descartado debido a que su exactitud da bajo el umbral requerido por el proyecto (0.75). Esto nos deja dos candidatos, El modelo Árbol de Decisión y el Modelo de Bosque Aleatorio.

Respecto de los datos que tenemos:

    - Modelo Árbol de Decisión: Calidad dataset validación:    0.785
                                Calidad dataset prueba:        0.810
                                Calidad dataset entrenamiento: 0.813

    - Modelo Bosque Aleatorio:  Calidad dataset validación:    0.769
                                Calidad dataset prueba:        0.772
                                Calidad dataset entrenamiento: 0.963
                                
Con esto a la vista, queda claro que el modelo más apropiado para seleccionar como el mejor para nuestro dataset completo es el de Árbol de Decisión. Si bien este posee una exactitud más baja para el conjunto de entrenamiento que para el modelo de Bosque Aleatorio, sus valores en los conjuntos de validación y prueba son excelentes en general. Los 3 dataset para el modelo árbol de decisión son muy similares entre sí, lo que señala que no hay un sobreajuste evidente a la vista, no así el caso para el modelo de Bosque Aleatorio.

Por lo anterior ya observado, nuestro modelo seleccionado será el Árbol de Decisión.
    

### Prueba de Cordura del Modelo Seleccionado (Árbol de Decisión)

Ya que es un modelo de clasificación, la prueba de cordura debe efectuarse comparando las predicciones efectuadas con la posibilidad aleatoria.



In [61]:
model = DecisionTreeClassifier(random_state=1702, max_depth = 5)
model.fit(features_train, target_train)
prediction = model.predict(features_test)
compare_table = pd.DataFrame(target_test).reset_index()
prediction = pd.Series(prediction)
compare_table["prediction"] = prediction
compare_table = compare_table.drop("index", axis = 1)
compare_table.head(20)
conteo = 0
for z in range(1,51):
    if compare_table["is_ultra"][z] != compare_table["prediction"][z]:
        conteo += 1
print("De las 50 primeras filas tenemos las siguientes diferencias:", conteo)


De las 50 primeras filas tenemos las siguientes diferencias: 8


Tomando aquello como muestra, si nuestro dataset fuesen solo aquellas filas, tenemos 42 predicciones correctas del dataset, lo que haciendo cálculos simples nos pone en un porcentaje de 80% de certeza. Muy similar a lo que nos señala nuestro modelo.

In [75]:
compare_table.sample(10)

,is_ultra,prediction
323,0,0
542,0,0
441,1,0
535,0,0
558,0,0
43,1,0
0,0,0
552,0,0
600,0,1
601,1,1


En este ejemplo tenemos en un total de 10 datos, 3 errores lo cual nos da un valor de 70% aproximado en su certeza, muy acorde a la realidad del modelo.

## Conclusiones

Habiendo concluido los 3 modelos, puedo señalar que efectivamente siempre dependerá de los datos que se tengan a disposición, la selección del mismo. En este caso sorprendió que el modelo Árbol de Decisión fuese el seleccionado por sobre el Bosque Aleatorio. Uno pensaría que ya que el bosque aleatorio genera muchas más instancias, sería más preciso en su búsqueda de una mejor exactitud, independiente de la velocidad (ya que el dataset no contenía tantos datos). Sin embargo, no fue el caso ya que si bien su exactitud era extremadamente alta en el conjunto de entrenamiento, no lo fue así en el de validación y prueba, lo que señala que seguramente se sobreajustó.


Claramente, Megaline debe utilizar para este dataset seleccionado, el modelo de Árbol de Decisión para efectuar la recomendación de planes Ultra o Smart en los clientes venideros. El modelo demostró ser el más robusto y parejo dentro de las exactitudes en los dataset de entrenamiento, validación y prueba.